In [10]:
import sf_quant.data as sfd
import sf_quant.optimizer as sfo
import sf_quant.backtester as sfb
import sf_quant.performance as sfp
import polars as pl
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm

In [2]:
df = pl.read_parquet('russell_3000_daily.parquet')

In [8]:
IC = 0.05

df = (
    df.lazy()
    .sort(["barrid", "date"])
    .with_columns([ # Convert nasty percents to nice fractions
        pl.col('specific_risk').truediv(100),
        pl.col('return').truediv(100),
        pl.col('specific_return').truediv(100)
    ])
    .with_columns(
        pl.col('return').log1p().alias('log_return')
    )
    .with_columns(
        pl.col("log_return")
            .rolling_sum(230)
            .over("barrid")
            .alias("momentum_temp")
    )
    .with_columns(
        pl.col("momentum_temp").shift(22).over("barrid").alias("momentum")
    )
    .with_columns(
        pl.col("log_return")
            .rolling_sum(22)
            .over("barrid")
            .alias("meanrev_temp")
    )
    .with_columns(
        (-pl.col("meanrev_temp").shift(1).over("barrid")).alias("meanrev")
    )
    .with_columns(
        (-pl.col("predicted_beta")).alias("bab")
    )
    .with_columns([ # Add signal z-scores
        ((pl.col("momentum") - pl.col("momentum").mean().over("date")) 
     / pl.col("momentum").std().over("date")).alias("momentum_z"),
        ((pl.col("meanrev") - pl.col("meanrev").mean().over("date")) 
     / pl.col("meanrev").std().over("date")).alias("meanrev_z"),
        ((pl.col("bab") - pl.col("bab").mean().over("date")) 
     / pl.col("bab").std().over("date")).alias("bab_z")
    ])
    .with_columns([ # Add signal alphas, using alpha = IC * specific_risk * z-score
        (IC * pl.col("specific_risk") * pl.col("momentum_z")).alias("momentum_alpha"),
        (IC * pl.col("specific_risk") * pl.col("meanrev_z")).alias("meanrev_alpha"),
        (IC * pl.col("specific_risk") * pl.col("bab_z")).alias("bab_alpha")
    ])
    .drop(["momentum_temp", "meanrev_temp"])
    .collect()
)

In [ ]:
# Need to add filter here to drop nulls, low prices, etc.

In [12]:
dates = df['date'].unique().sort().to_list()

constraints = [
    sfo.FullInvestment(),
    sfo.LongOnly(),
    sfo.NoBuyingOnMargin(),
    sfo.UnitBeta()
]

portfolio_list = []
for date_ in tqdm.tqdm(dates, "Computing portfolios"):
    subset = (
        df.filter(
            pl.col('date').eq(date_)
        )
        .sort('barrid')
    )
    barrids = subset['barrid'].to_list()
    alphas_np = subset['momentum_alpha'].to_numpy()
    betas_np = subset['predicted_beta'].to_numpy()

    cov_mat = sfd.construct_covariance_matrix(date_, barrids).drop('barrid').to_numpy()

    portfolio = sfo.mve_optimizer(
        ids=barrids,
        alphas=alphas_np,
        covariance_matrix=cov_mat,
        constraints=constraints,
        gamma=10,
        betas=betas_np
    )

    portfolio = portfolio.with_columns(
        pl.lit(date_).alias('date')
    )

    portfolio_list.append(portfolio)

weights = pl.concat(portfolio_list)

Computing portfolios:   0%|          | 0/7602 [02:23<?, ?it/s]

Solver interrupted


SolverError: Solver 'OSQP' failed. Try another solver, or solve with verbose=True for more information.

In [ ]:
weights

In [9]:
df

date,barrid,ticker,price,return,specific_return,specific_risk,historical_beta,predicted_beta,market_cap,daily_volume,bid_ask_spread,log_return,momentum,meanrev,bab,momentum_z,meanrev_z,bab_z,momentum_alpha,meanrev_alpha,bab_alpha
date,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2013-07-31,"""USA06Z1""","""MDXG""",6.26,-0.000016,-0.000079,0.005506,0.328385,0.34349,6.006157e8,121693.0,0.01,-0.000016,null,null,-0.34349,null,null,2.149267,null,null,0.000592
2013-08-01,"""USA06Z1""","""MDXG""",6.32,0.000096,0.0000365,0.005503,0.334989,0.353329,6.0865392e8,131728.0,0.01,0.000096,null,null,-0.353329,null,null,2.111728,null,null,0.000581
2013-08-02,"""USA06Z1""","""MDXG""",6.31,-0.000016,-0.000073,0.005481,0.330713,0.363624,6.0769086e8,43252.0,0.01,-0.000016,null,null,-0.363624,null,null,2.086144,null,null,0.000572
2013-08-05,"""USA06Z1""","""MDXG""",6.45,0.000222,0.0001936,0.005477,0.324494,0.356596,6.211737e8,70944.0,0.02,0.000222,null,null,-0.356596,null,null,2.098288,null,null,0.000575
2013-08-06,"""USA06Z1""","""MDXG""",6.29,-0.000248,-0.000053,0.005469,0.366323,0.399196,6.0576474e8,77085.0,0.01,-0.000248,null,null,-0.399196,null,null,2.001238,null,null,0.000547
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-09-09,"""USBRKA2""","""GLIBA""",36.225,-0.000043,0.0001121,0.002084,0.236481,0.302514,1.3226e8,31025.0,0.11,-0.000043,null,-0.000583,-0.302514,null,0.100295,2.013268,null,0.00001,0.00021
2025-09-10,"""USBRKA2""","""GLIBA""",36.24,0.000004,0.000231,0.002095,0.252015,0.29845,1.3231e8,6298.0,0.21,0.000004,null,-0.000102,-0.29845,null,0.428666,2.007619,null,0.000045,0.00021
2025-09-11,"""USBRKA2""","""GLIBA""",37.05,0.000224,0.0000213,0.002148,0.256752,0.31384,1.3527e8,26857.0,0.25,0.000223,null,-0.000229,-0.31384,null,0.301627,1.987568,null,0.000032,0.000213
